In [41]:
import pandas as pd

df = pd.DataFrame({'A': {0: 'a', 1: 'b', 2: 'c'},
                   'B': {0: 1, 1: 3, 2: 5},
                   'C': {0: 2, 1: 4, 2: 6}})
df

,A,B,C
0,a,1,2
1,b,3,4
2,c,5,6


In [42]:
pd.melt(df, value_vars=['B'])

,variable,value
0,B,1
1,B,3
2,B,5


In [43]:
pd.melt(df, value_vars=['B', 'C'])

,variable,value
0,B,1
1,B,3
2,B,5
3,C,2
4,C,4
5,C,6


In [44]:
pd.melt(df, id_vars=['A'], value_vars=['B'])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5


In [45]:
melt_ = pd.melt(df, id_vars=['A'], value_vars=['B', 'C'])
melt_

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5
3,a,C,2
4,b,C,4
5,c,C,6


In [46]:
melt_.pivot(index='A', columns='variable', values='value').reset_index()

variable,A,B,C
0,a,1,2
1,b,3,4
2,c,5,6


In [47]:
pd.melt(df, id_vars=['A'], value_vars=['B'],
         var_name='myVarname', value_name='myValname')

,A,myVarname,myValname
0,a,B,1
1,b,B,3
2,c,B,5


In [48]:
df.columns = [list('AAC'), list('DEF')]
df

A     C
   D  E  F
0  a  1  2
1  b  3  4
2  c  5  6

In [49]:
pd.melt(df, id_vars=[('A', 'D')], value_vars=[('C', 'F')])

,"(A, D)",variable_0,variable_1,value
0,a,C,F,2
1,b,C,F,4
2,c,C,F,6


In [50]:
mu_melt = pd.melt(df, id_vars=[('A', 'E')], value_vars=[('A', 'D'), ('C', 'F')]) # 分层索引的melt操作
mu_melt

,"(A, E)",variable_0,variable_1,value
0,1,A,D,a
1,3,A,D,b
2,5,A,D,c
3,1,C,F,2
4,3,C,F,4
5,5,C,F,6


In [51]:
mu_melt.columns

Index([('A', 'E'), 'variable_0', 'variable_1', 'value'], dtype='object')

In [52]:
# 列名为元组,不能通过列名进行索引===>避免列名为元组
mu_melt.loc[:, mu_melt.columns[0]]

KeyError: "None of [Index(['A', 'E'], dtype='object')] are in the [columns]"